In [1]:
import os
from crewai import Agent, Task, Crew

if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError(
        "OPENAI_API_KEY not set. "
        "Add it as a GitHub Codespaces secret or local env variable."
    )

print("✅ Environment ready")

✅ Environment ready
